In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-annotated/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'label','commit_message':'message','commit_diff':'diff'})
df = df.replace({"label": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_4560/1556623075.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,label,diff,message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_DEL_LINE,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE
commit_sha,,,,,,,,,,,,,,,,,,,,,
be937a3290223f926fe50684f1344569a573ed4b,1,diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
c800d2e36954edddcb83aa1df7f623f2780c7780,0,diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
469e56eeff17eb857c95e935ca7b49723c43470e,2,diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...,0,0,0,0,0,0,0,...,0,0,8,0,0,15,0,0,0,0
24f5d302c0906f3377f4d6bcc2735d943ce13e3d,0,diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,...",3,37,0,0,9,0,0,...,0,0,0,0,0,0,1,2,0,0
1effdb7b959503596b07f7f3e74618ab63f9e5e5,2,diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...,1,0,12,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,1,diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,1,diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...,2,6,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,1,diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...,1,1,5,13,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-annotated/eval.csv',index=False)

In [5]:
train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=1, random_state=42))

/tmp/ipykernel_4560/3367951317.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby("label", group_keys=False).apply(lambda x: x.sample(n=1, random_state=42))


In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train

,label,diff,message,CODE_CHANGE_FILE,CODE_ADD_CODE_LINE,CODE_DEL_CODE_LINE,CODE_MOD_CODE_LINE,CODE_ADD_COMMENT_LINE,CODE_DEL_COMMENT_LINE,CODE_MOD_COMMENT_LINE,...,BUILD_DEL_LINE,BUILD_MOD_LINE,DOC_CHANGE_FILE,DOC_ADD_LINE,DOC_DEL_LINE,DOC_MOD_LINE,OTHER_CHANGE_FILE,OTHER_ADD_LINE,OTHER_DEL_LINE,OTHER_MOD_LINE
commit_sha,,,,,,,,,,,,,,,,,,,,,
ed09b4ea6a17566b546a9d3ae76876636710c6a0,0,diff --git a/common/net/json/host.go b/common/...,support domain as host in VMess outbound confi...,3,49,2,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a2479f46f3d05b37254ad701b6b76f84624d3cb5,1,diff --git a/django/contrib/auth/migrations/00...,Fixed #7220 -- Allowed AbstractBaseUser.last_l...,3,13,0,3,1,0,0,...,0,0,2,17,0,1,0,0,0,0
de7140853140029a3f48600b60e700464e7662b5,2,diff --git a/packages/plugin-legacy/index.js b...,fix(plugin-legacy): avoid executing blank dyna...,1,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)


base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=10, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 1.1536: 100%|██████████| 1/1 [00:00<00:00,  4.44batch/s]


=============================train========================


Epoch 2/10 Loss: 0.9051: 100%|██████████| 1/1 [00:00<00:00,  8.52batch/s]


=============================train========================


Epoch 3/10 Loss: 0.7698: 100%|██████████| 1/1 [00:00<00:00,  8.70batch/s]


...

=============================train========================


Epoch 9/10 Loss: 0.1197: 100%|██████████| 1/1 [00:00<00:00,  8.26batch/s]


=============================train========================


Epoch 10/10 Loss: 0.1255: 100%|██████████| 1/1 [00:00<00:00,  8.54batch/s]


=============================train========================
Validation Accuracy: 0.2911
Precision: 0.4203
Recall: 0.2911
F1-Score: 0.2960
              precision    recall  f1-score   support

           0     0.1176    0.2353    0.1569        34
           1     0.2696    0.5167    0.3543        60
           2     0.5556    0.2098    0.3046       143

    accuracy                         0.2911       237
   macro avg     0.3143    0.3206    0.2719       237
weighted avg     0.4203    0.2911    0.2960       237



In [9]:
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.2941
Precision: 0.4417
Recall: 0.2941
F1-Score: 0.3124
              precision    recall  f1-score   support

           0     0.1702    0.2500    0.2025        32
           1     0.1855    0.4510    0.2629        51
           2     0.5821    0.2516    0.3514       155

    accuracy                         0.2941       238
   macro avg     0.3126    0.3175    0.2722       238
weighted avg     0.4417    0.2941    0.3124       238



In [11]:
from sklearn import metrics

In [12]:
cm = metrics.confusion_matrix(test_labels, test_predictions)
cm

array([[ 8, 13, 11],
       [11, 23, 17],
       [28, 88, 39]])